In [26]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [27]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [28]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [29]:
song_files = get_files('data/song_data')

In [30]:
filepath = song_files[0]

In [31]:
df = pd.read_json(filepath,lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


In [32]:
song_data=df.loc[:,['song_id','title','artist_id','year','duration']].values.tolist()[0]

In [33]:
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [34]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [35]:
artist_data = get_files('artist_data')
filepath = song_files[0]

df = pd.read_json(filepath,lines=True)
df.values.tolist()[0]

df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


In [36]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [37]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [38]:
log_files = get_files('data/log_data')

In [39]:
filepath = log_files[0]
filepath

'/home/workspace/data/log_data/2018/11/2018-11-30-events.json'

In [40]:
df =pd.read_json(filepath,lines=True)
df.values.tolist()[0]
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [41]:
df=df[df.page=='NextSong']  


In [42]:
df['ts'] = pd.to_datetime(df['ts'])
df['ts'] = df['ts'].dt.floor('min')

In [43]:
time_data= df['ts']
time_data.head()

0   1970-01-01 00:25:00
1   1970-01-01 00:25:00
2   1970-01-01 00:25:00
3   1970-01-01 00:25:00
4   1970-01-01 00:25:00
Name: ts, dtype: datetime64[ns]

In [56]:

column_labels = (time_data,time_data.dt.year,time_data.dt.month,time_data.dt.day,time_data.dt.hour,
                 time_data.dt.dayofweek,time_data.dt.dayofyear)

In [57]:
lables=('upto','Year','Month','Day','Hour','Day_Of_the_week','Day_of_the_Year')

In [58]:
time_df = pd.DataFrame.to_dict(dict(zip(lables,column_labels)))
time_df.head()

AttributeError: 'dict' object has no attribute 'columns'

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [46]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

NameError: name 'time_df' is not defined

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [101]:
user_df =df.loc[:,['userId','firstName','lastName','gender','level']]
user_df

,userId,firstName,lastName,gender,level
0,91,Jayden,Bell,M,free
1,73,Jacob,Klein,M,paid
2,73,Jacob,Klein,M,paid
3,73,Jacob,Klein,M,paid
4,73,Jacob,Klein,M,paid
5,73,Jacob,Klein,M,paid
6,73,Jacob,Klein,M,paid
7,73,Jacob,Klein,M,paid
9,73,Jacob,Klein,M,paid
10,73,Jacob,Klein,M,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [27]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

NameError: name 'user_df' is not defined

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [28]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

AttributeError: 'Series' object has no attribute 'song'

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [29]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.